In [1]:
import tensorflow as tf
import os
import time

from model import *
from data_utils import *
from conf_utils import *

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class config:
    vocab_size = 2000
    embedding_size = 200
    filter_size = 64
    num_layers = 5
    block_size = 2
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 16
    epochs = 5
    num_sampled = 64
    learning_rate = 0.0001
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    #data_dir = "data/texts/reviews/movie_reviews"
    data_dir = "data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled"

In [3]:
#with open("data/texts/reviews/movie_reviews") as f:
    #text = f.readlines()

In [4]:
conf = prepare_conf(config)

In [5]:
conf.text_size

22

In [6]:
x_batches, y_batches = prepare_data(conf)

In [7]:
x_batches[0][0]

array([  1,   1,   2,  67,   0,   6,  57,   0, 611, 134, 650,   7,   0,
        46,  14,   9,   0, 427,  31, 785,   4,   3])

In [8]:
conf.num_batches

60069

In [9]:
model = GatedCNN(conf)
saver = tf.train.Saver(tf.trainable_variables())
print("Started Model Training...")

Started Model Training...


In [11]:
batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(conf.summary_path, graph=sess.graph)

    if os.path.exists(conf.ckpt_file):
        saver.restore(sess, conf.ckpt_file)
        print("Model Restored")

    for i in np.arange(1):
        start = time.time()
        #for j in np.arange(conf.num_batches):
        for j in np.arange(21):
            inputs, labels, batch_idx = get_batch(x_batches, y_batches, batch_idx)
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.y:labels})
            if j%20 == 0:
                print(l)
        end = time.time()
        print("Epoch: %.2f, Time: %.2f,  Loss: %.2f"%(i, end-start, l))

        if i % 3 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.y:labels})
            print("Perplexity: %.2f"%perp)
            saver.save(sess, conf.ckpt_file)

        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.y:labels})
        summary_writer.add_summary(summaries, i)

2581739.2
2589265.0
Epoch: 0.00, Time: 99.48,  Loss: 2589265.00
Perplexity: inf


In [12]:
with tf.Session() as sess:
    saver.restore(sess, conf.ckpt_file)
    #Get the contextualized representation
    out_layer = sess.run(model.out_layer, feed_dict={model.X:inputs})
    
    #out_layer.resha

INFO:tensorflow:Restoring parameters from ckpt/vocab2000_embed200_filters64_batch16_layers5_block2_fdim5/model.ckpt


In [13]:
out_layer.shape

(16, 22, 200)

In [9]:
from nltk.corpus import movie_reviews
reviews = open('movie_reviews', 'w+')
for file in movie_reviews.fileids():
    words = movie_reviews.words(file)
    if len(words) > config.context_size-2:
        words = words[:(config.context_size-2)]
        text = ' '.join(words)
        reviews.write(text+'\n')
reviews.close()